# Notebook 699: Tarea calificada 2, INAR 23-24

## Generación de texto seq2seq model
## A partir de textos de parlamentarios españoles (anteriores a 2022)

## Nota importante

Esta tarea en su versión 2023-24 surge del excelente trabajo de varios compañeros del curso 2022-23, que aunque yo proporcioné un dataset de textos a partir de las intervenciones de parlamentarios (los líderes de varios partidos en 2021-22, alguno de los cuales ya no está en la política española), hicieron un extraordinario "escrapeo" de la web del Congreso de los Diputados y enriquecieron de forma notable el dataset. Este es el que propongo para esta tarea.

Debo decir que si hay un texto (o lenguaje natural) libre de derechos y especialmente actual, son las intervenciones (estrictamente **públicas**) de los representantes elegidos en elecciones, y que el Congreso debería facilitar, no ya para su uso en estas tareas, sino para cualquier estudioso del español, o de la política, o de la psicología de los políticos.

Por supuesto, esto son opiniones estrictamete mías, en el momento concreto en que las escribo, y sencillamente quiero hacer homenaje a los que colaboraron tanto con este trabajo que espero encontréis interesante.

## ¿De qué trata esta tarea?

Pues ni más ni menos que de generar texto en español a partir de texto de parlamentarios, basado en el tutorial que hemos seguido en clase:

https://www.tensorflow.org/text/tutorials/text_generation?hl=es-419

Para facilitar la tarea se propone un pre-proceso (basado en la tarea 2021-22), y la tarea se concreta en el modelo para generar texto y en las pruebas de la calidad del texto generado.


## Calificación

Está explicada en la entrada correspondiente de Blackboard. Básicamente, hay un mínimo que consiste en proponer tres modelos de red recurrente, uno para cada parlamentario, entrenarlos, y **evaluarlos** generando texto y comentando su calidad.

Para llegar a la máxima nota, propongo poner a dialogar los tres modelos.

Pero por supuesto, valoraré el trabajo de construcción del modelo. Para esta tarea no hay una "medida" como la accuracy en la tarea 1. Será relativamente subjetiva. Por eso parece aconsejable comenzar con modelos pequeños o con pocas etapas e ir refinando.

## Setup

Para facilitar la tarea propongo unas cuantas casillas para cargar en memoria los textos, tres .txt que están incluidos en un .zip.

## Nota importante

La codificación (juego de caracteres) es UTF-8 y creo que debe seguir siendo así. *NO* abráis los .txt con el Notepad de Windows, sino con el Notepad+++ que os permitiría cambiarlo o devolverlo a UTF-8 (o Unicode si queréis).

A pesar que la salida por pantalla (en mi sistema, un Linux) de caracteres ñ y acentuados parece que está mal, luego la generación de texto (insisto, lo he comprobado en mi sistema) es correcta en español.


### Import TensorFlow and other libraries

In [1]:
import tensorflow as tf
import numpy as np
import os
import time
import re
import os


2023-12-13 12:55:10.741909: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-13 12:55:10.741984: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-13 12:55:10.744464: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-13 12:55:10.759278: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-13 12:55:12.734532: W tensorflow/compiler/tf2

## Lectura de ficheros de datos

In [2]:
datos_abascal   = "intervencionesAbascal.txt"
datos_sanchez   = "intervencionesSanchez.txt"
datos_casado    = "intervencionesCasado.txt"

### Leer los ficheros de datos

Primero, abrimos el texto de Santiago Abascal, que es el más corto, y lo leemos:

In [3]:
# Read, then decode for py2 compat.
text = open(datos_abascal, 'rb').read().decode(encoding='utf-8')

# length of text is the number of characters in it
print(f'Texto de Santiago Abascal: {len(text)} carácteres')

Texto de Santiago Abascal: 22573 carácteres


In [4]:
# Take a look at the first 250 characters in text
print(text[:250])

Señor Sánchez, ¿cómo se atreve usted a hablarme de monólogos si siempre trae las respuestas escritas, si usted nunca contesta a mis preguntas? Conteste por lo menos hoy. ¿Qué va a hacer usted para impedir que VOX siga cruzando las líneas que dice ust


In [5]:
# The unique characters in the file
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')

81 unique characters


Vamos a ver cual de los tres textos tiene el mayor vocabulario, para usar el mismo en los tres modelos:

In [6]:
vocab_mayor = vocab

textos = []

for texto in [datos_abascal, datos_sanchez, datos_casado]:
    text = open(texto, 'rb').read().decode(encoding='utf-8')
    vocab = sorted(set(text))
    print(f'{len(text)} carácteres, {len(vocab)} únicos en {texto}')
    
    if len(vocab) > len(vocab_mayor):
        vocab_mayor = vocab
    
    textos.append(text)
        
vocab = sorted(set(textos[0] + textos[1] + textos[2]))
print(f'{len(vocab)} únicos en los tres textos')

22573 carácteres, 81 únicos en intervencionesAbascal.txt
239623 carácteres, 104 únicos en intervencionesSanchez.txt
105940 carácteres, 92 únicos en intervencionesCasado.txt
108 únicos en los tres textos


## Procesar el texto

### Vamos a vectorizar el texto

Como las redes neuronales no entienden carácteres sino números, vamos a vectorizar el texto. Para ello, vamos a crear dos *"tablas de traducción"*, uno para pasar de carácter a número y otro para pasar de número a carácter.

In [7]:
# Creamos un diccionario para asignar cada caracter a un entero
char2idx = {u:i for i, u in enumerate(vocab)}

# Luego hacemos una lista con los carácteres ordenados por su entero
idx2char = np.array(vocab)

# Vamos a ver que pinta tiene nuestro diccionario
for char,_ in zip(char2idx, range(20)):
    print(f'{repr(char)}: {char2idx[char]}')

'\x07': 0
'\n': 1
' ': 2
'!': 3
'%': 4
'&': 5
'(': 6
')': 7
',': 8
'-': 9
'.': 10
'0': 11
'1': 12
'2': 13
'3': 14
'4': 15
'5': 16
'6': 17
'7': 18
'8': 19


Ahora ya podemos vectorizar el texto

In [8]:
# Ahora podemos convertir todo el texto a enteros
text_as_int = np.array([char2idx[c] for c in text])

# Vamos a ver como queda el texto en enteros
print(f'{repr(text[:13])} ---- carácteres mapeados a int ----> {text_as_int[:13]}')

'Señor Sánchez' ---- carácteres mapeados a int ----> [42 56 93 66 69  2 42 90 65 54 59 56 77]


### The prediction task

Given a character, or a sequence of characters, what is the most probable next character? This is the task you're training the model to perform. The input to the model will be a sequence of characters, and you train the model to predict the output—the following character at each time step.

Since RNNs maintain an internal state that depends on the previously seen elements, given all the characters computed until this moment, what is the next character?


# Fases propuestas para la elaboración del modelo

### 1. Crear los training examples y los targets

Ahora vamos a divir nuestro texto en secuencia de carácteres. Cada secuencia tendrá `seq_length` carácteres de nuestro texto.
Para cada secuencia de entrada, los targets correspondientes contienen la misma longitud de texto, excepto desplazada un carácter a la derecha.
Por eso dividimos el texto en secuencias de `seq_length+1`. Por ejemplo, digamos que `seq_length` es 4 y nuestro texto es "Hola". La secuencia de entrada sería "Hol" y la secuencia de salida "ola".

Para hacer esto, primero usamos la función `tf.data.Dataset.from_tensor_slices` para convertir el vector de texto en una secuencia de índices de caracteres.



In [9]:
# Creamos un dataset de tensorflow con los enteros
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

# Ahora vamos a dividir el texto en secuencias de 100 carácteres
SEQ_LENGTH = 100
sequences = char_dataset.batch(SEQ_LENGTH + 1, drop_remainder=True)

# Vamos a ver como son estas secuencias
print("Secuencias de 100 carácteres:")
for item in sequences.take(5):
    print(repr(''.join(idx2char[item.numpy()])))

Secuencias de 100 carácteres:
'Señor Sánchez, sus recetas económicas son tan creíbles como sus promesas electorales, y encima propon'
'en las mismas recetas fracasadas que nos llevaron a la peor crisis económica de nuestra historia: más'
' despilfarro, más déficit y más impuestos. Pero el Partido Popular es un partido de Estado y también '
'de Gobierno, aunque estemos temporalmente en la oposición. Por eso el lunes le ofrecí pactar los Pres'
'upuestos Generales si rompe con los independentistas, una oferta, por cierto, a la que usted no ha co'


2023-12-13 12:55:15.524950: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-12-13 12:55:15.771591: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-12-13 12:55:15.771706: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-12-13 12:55:15.786174: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-12-13 12:55:15.786338: I external/local_xla/xla/stream_executor

### 2. Crear los training batches

El conjunto de datos de entrenamiento contiene tanto los datos de entrada (desde la posición 0 a la 99) como los de salida (desde la posición 1 a la 100). Por lo que necesitamos mapear el input y el target para crear el dataset.

In [10]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

# Ahora vamos a aplicar la función anterior a todas las secuencias
dataset = sequences.map(split_input_target)

# Vamos a ver como son las secuencias de entrada y salida
for input_example, target_example in  dataset.take(1):
    print ('Input: ', repr(''.join(idx2char[input_example.numpy()])))
    print ('Target: ', repr(''.join(idx2char[target_example.numpy()])))
    

Input:  'Señor Sánchez, sus recetas económicas son tan creíbles como sus promesas electorales, y encima propo'
Target:  'eñor Sánchez, sus recetas económicas son tan creíbles como sus promesas electorales, y encima propon'


Ahora ya podemos mezclar los datos y empaquetarlos en batches de 64 secuencias.

In [11]:
BATCH_SIZE = 64
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
)

dataset

<_BatchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

## 3. Crear el modelo

Puedes usar cualquiera de los modelos (RNN, LSTM, GRU) que hemos visto en clase. Por supuesto, del tamaño del modelo (capas, neuronas en cada capa) así como de las épocas (más adelante) dependerá el tiempo de proceso en el .fit

Para el modelo de Abascal vamos a usar una RNN que contenga solo una capa LSTM. En concreto, definiremos una red neuronal de solo 3 capas:

- Capa de entrada: una capa de tipo Embedding, que convierte los índices de los caracteres en vectores embedding de tamaño embedding_dim. En las opciones de la capa especificaremos el tamaño de nuestro vocabulario `(vocab_size)` y el tamaño de los vectores embedding `(embedding_dim)`. También indicaremos el tamaño del batch que vamos a usar `(batch_size)`.

- Capa LSTM: una capa LSTM con `units=2048`, que es el número de neuronas recurrentes de la capa. También indicaremos con return_sequences=True que queremos predecir el carácter siguiente a todos los carácteres de entrada y no solo al último carácter. El argumento `stateful=True` explica el uso de las capacidades de memoria de la red entre batches: Si está en False, por cada nuevo batch se inicializan las memory cells (la parte de la red neuronal que preserva el estado de la red a través del tiempo), pero si está en True, por cada nuevo batch se mantienen las memory cells con las actualizaciones hechas durante la ejecución del batch anterior. El último argumento, `recurrent_initializer='glorot_uniform'`, es un que indica como se inicializan los pesos de las matrices internas de la capa LSTM. En estos casosm la distribución más común es la `glorot_uniform`.

- Capa de salida: una capa Dense con `vocab_size` neuronas. Esta capa nos dará como salida un vector de tamaño `vocab_size` con las probabilidades de que el siguiente carácter sea cada uno de los carácteres del vocabulario.

In [12]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense

def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = Sequential([
        Embedding(vocab_size, embedding_dim,
                  batch_input_shape=[batch_size, None]),
        LSTM(rnn_units, return_sequences=True,
             recurrent_initializer='glorot_uniform',
             stateful=True),
        Dense(vocab_size)
    ])
    
    return model

vocab_size = len(vocab)
embedding_dim = 512
rnn_units = 2048

model = build_model(
    vocab_size=len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)

model.summary()
 

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (64, None, 512)           55296     
                                                                 
 lstm (LSTM)                 (64, None, 2048)          20979712  
                                                                 
 dense (Dense)               (64, None, 108)           221292    
                                                                 
Total params: 21256300 (81.09 MB)
Trainable params: 21256300 (81.09 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


Definimos nuestra función de pérdida y el optimizador que vamos a usar para entrenar el modelo. En este caso, usaremos la función de pérdida `sparse_categorical_crossentropy` y el optimizador `Adam` con sus argumentos por defecto. Con esto ya podemos compilar el modelo.

In [13]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

In [14]:
model.compile(optimizer='adam', loss=loss)

Podemos usar la técnica de los checkpoints para no perder el progreso del entrenamiento si tenemos un fallo en el sistema. El único problema es que los checkpoints pueden llegar a ocupar mucho espacio muy rápidamente, por lo que es recomendable borrarlos después de entrenarlos y en su lugar guardar el modelo ya terminado.

In [15]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

## 4. Summary y fit del modelo



In [16]:
EPOCHS = 200

history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

# Guardamos el modelo y los pesos
model.save_weights('./modelos/abascal_weights.keras')

Epoch 1/200


2023-12-13 12:55:20.045617: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8902
Could not load symbol cublasGetSmCountTarget from libcublas.so.11. Error: /home/antonio/.anaconda3/envs/tf/lib/./libcublas.so.11: undefined symbol: cublasGetSmCountTarget
2023-12-13 12:55:20.167953: I external/local_xla/xla/service/service.cc:168] XLA service 0x7f2a742b01c0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-12-13 12:55:20.168047: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3060 Laptop GPU, Compute Capability 8.6
2023-12-13 12:55:20.178246: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1702468520.310589   83645 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


16/16 [==============================] - 8s 258ms/step - loss: 3.9156
Epoch 2/200
16/16 [==============================] - 4s 257ms/step - loss: 2.9504
Epoch 3/200
16/16 [==============================] - 4s 255ms/step - loss: 2.7219
Epoch 4/200
16/16 [==============================] - 4s 219ms/step - loss: 2.4696
Epoch 5/200
16/16 [==============================] - 4s 261ms/step - loss: 2.3360
Epoch 6/200
16/16 [==============================] - 4s 262ms/step - loss: 2.2489
Epoch 7/200
16/16 [==============================] - 6s 353ms/step - loss: 2.1799
Epoch 8/200
16/16 [==============================] - 5s 327ms/step - loss: 2.1150
Epoch 9/200
16/16 [==============================] - 4s 259ms/step - loss: 2.0454
Epoch 10/200
16/16 [==============================] - 4s 256ms/step - loss: 1.9866
Epoch 11/200
16/16 [==============================] - 4s 266ms/step - loss: 1.9259
Epoch 12/200
16/16 [==============================] - 5s 281ms/step - loss: 1.8569
Epoch 13/200
16/16 [=====

Ahora que hemos terminado de entranar el modelo ya no necesitamos los chekpoints, por lo que podemos borrarlos.

In [17]:
# Borramos los checkpoints para no ocupar espacio

import os, shutil
folder = './training_checkpoints/'

for filename in os.listdir(folder):
    file_path = os.path.join(folder, filename)
    try:
        if os.path.isfile(file_path) or os.path.islink(file_path):
            os.unlink(file_path)
        elif os.path.isdir(file_path):
            shutil.rmtree(file_path)
    except Exception as e:
        print('Failed to delete %s. Reason: %s' % (file_path, e))

## 5. Genera texto y evalúa su calidad

Para generar texto a partir del modelo, ahora necesitamos un `batch_size` de 1, por lo que tenemos que rehacer el modelo y cargar los pesos de nuestro modelo entrenado.

In [18]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights('./modelos/abascal_weights.keras')
model.build(tf.TensorShape([1, None]))

model.save('./modelos/abascal.keras')

In [19]:
def generate_text(model, start_string):
    
    num_generate = 500
    input_eval = [char2idx[s] for s in start_string]
    
    input_eval = tf.expand_dims(input_eval, 0)
    text_generated = []
    temperature = 0.5
    
    model.reset_states()
    
    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)
        
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()
        
        input_eval = tf.expand_dims([predicted_id], 0)
        
        text_generated.append(idx2char[predicted_id])
        
    return (''.join(text_generated))

In [20]:
print(generate_text(model, start_string=u"Señor Abascal, ¿que opina del precio de la luz?"))


 ¿No va a destruir a ministros, a barones, también a sus socios de Gobierno, al Consejo de Estado, y hasta a los jueces y las pensiones; que no trabaje para su fragmentación, sino para ser admirada en el mundo; que no se puede agredir así a las instituciones con la complicidad de este Gobierno. No todo vale, basta y que hay que apedrearle  solo porque sus padres hayan pedido que se cumplan los derechos en la cárcel le vuelve a hacer sentarse en una mesa por la autodeterminación. Y para su vergüe


## 6. Trabajo adicional

Por ejemplo, poner en cadena los tres modelos para que "dialoguen" entre sí

### Modelo de Pedro Sánchez

Creamos el dataset de entrenamiento de Pedro Sánchez

In [21]:
# Abrir el texto
text = open(datos_sanchez, 'rb').read().decode(encoding='utf-8')

# Vectorizar el texto
text_as_int = np.array([char2idx[c] for c in text])

# Secuencias de 100 carácteres
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)
SEQ_LENGTH = 100
sequences = char_dataset.batch(SEQ_LENGTH + 1, drop_remainder=True)

# Dataset de entrenamiento
dataset = sequences.map(split_input_target)
BATCH_SIZE = 64
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
)

Ahora creamos el modelo de Pedro Sánchez

In [22]:
model = build_model(
    vocab_size=len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)

model.compile(optimizer='adam', loss=loss)

Entrenamos el modelo de Pedro Sánchez

In [23]:
EPOCHS = 200

history = model.fit(dataset, epochs=EPOCHS)

# Guardamos el modelo y los pesos
model.save_weights('./modelos/sanchez_weights.keras')

Epoch 1/200
37/37 [==============================] - 11s 261ms/step - loss: 3.3824
Epoch 2/200
37/37 [==============================] - 10s 267ms/step - loss: 2.5508
Epoch 3/200
37/37 [==============================] - 10s 269ms/step - loss: 2.2240
Epoch 4/200
37/37 [==============================] - 10s 267ms/step - loss: 2.0462
Epoch 5/200
37/37 [==============================] - 10s 264ms/step - loss: 1.8751
Epoch 6/200
37/37 [==============================] - 10s 259ms/step - loss: 1.7157
Epoch 7/200
37/37 [==============================] - 6s 166ms/step - loss: 1.5714
Epoch 8/200
37/37 [==============================] - 5s 133ms/step - loss: 1.4418
Epoch 9/200
37/37 [==============================] - 11s 284ms/step - loss: 1.3399
Epoch 10/200
37/37 [==============================] - 11s 279ms/step - loss: 1.2468
Epoch 11/200
37/37 [==============================] - 9s 243ms/step - loss: 1.1700
Epoch 12/200
37/37 [==============================] - 9s 248ms/step - loss: 1.1055
Epoch

Creamos el modelo para generar texto de Pedro Sánchez

In [24]:
vocab_size = len(vocab)

model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights('./modelos/sanchez_weights.keras')
model.build(tf.TensorShape([1, None]))

model.save('./modelos/sanchez.keras')

Comprobamos el texto generado por Pedro Sánchez

In [25]:
print(generate_text(model, start_string=u"Que opina del señor Abascal?"))

  Fíjense, señor Casado, que es la unidad del conjunto de las fuerzas de esta Cámara, sobre todo para poner en marcha —de corazón se la pasada más responsabilidad de Pala comunidad autónoma más afectada hasta ahora por la particas, al menos, señoría, yo tengo más claro que se haganes de euros. Es verdad que el tramo cuarto, que se refiere precisamente a le diré una cosa: ustedes han representado la continuidad y el cambio; la continuidad en las malas a través del esfuerzo de las ayudas de CO2, e


### Modelo de Pablo Casado (GRU)

Creamos el dataset de entrenamiento de Pablo Casado

In [26]:
# Abrir el texto
text = open(datos_casado, 'rb').read().decode(encoding='utf-8')

# Vectorizar el texto
text_as_int = np.array([char2idx[c] for c in text])

# Secuencias de 100 carácteres
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)
SEQ_LENGTH = 100
sequences = char_dataset.batch(SEQ_LENGTH + 1, drop_remainder=True)

# Dataset de entrenamiento
dataset = sequences.map(split_input_target)
BATCH_SIZE = 64
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
)

Creamos el modelo de Pablo Casado, como lo haremos con GRU tendremos que hacer una nueva implementación de model.

In [32]:
class gru_model(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [34]:
model = gru_model(
    # Be sure the vocabulary size matches the `StringLookup` layers.
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

model.compile(optimizer='adam', loss=loss)

Entrenamos el modelo de Pablo Casado

In [35]:
EPOCHS = 200

history = model.fit(dataset, epochs=EPOCHS)

Epoch 1/200
16/16 [==============================] - 8s 139ms/step - loss: 5.2356
Epoch 2/200
16/16 [==============================] - 2s 134ms/step - loss: 2.9608
Epoch 3/200
16/16 [==============================] - 2s 135ms/step - loss: 2.5745
Epoch 4/200
16/16 [==============================] - 2s 135ms/step - loss: 2.3653
Epoch 5/200
16/16 [==============================] - 2s 138ms/step - loss: 2.2457
Epoch 6/200
16/16 [==============================] - 2s 133ms/step - loss: 2.1553
Epoch 7/200
16/16 [==============================] - 2s 126ms/step - loss: 2.0795
Epoch 8/200
16/16 [==============================] - 2s 132ms/step - loss: 2.0068
Epoch 9/200
16/16 [==============================] - 2s 137ms/step - loss: 1.9351
Epoch 10/200
16/16 [==============================] - 2s 138ms/step - loss: 1.8646
Epoch 11/200
16/16 [==============================] - 2s 137ms/step - loss: 1.7938
Epoch 12/200
16/16 [==============================] - 2s 137ms/step - loss: 1.7229
Epoch 13/200


Para poder generar texto con el nuevo modelo de GRU tendremos que crear una clase nueva de OneStep.

In [37]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [42]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

one_step_model.save('./modelos/casado.keras')

Comprobamos el texto generado por Pablo Casado

In [41]:
start = time.time()
states = None
next_char = tf.constant(['CASADO:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)


ValueError: in user code:

    File "/tmp/ipykernel_83518/1970756946.py", line 33, in generate_one_step  *
        predicted_logits = predicted_logits + self.prediction_mask

    ValueError: Dimensions must be equal, but are 108 and 109 for '{{node add}} = AddV2[T=DT_FLOAT](truediv, add/y)' with input shapes: [1,108], [109].


### Conversación entre Pedro Sánchez, Pablo Casado y Santiago Abascal

Primero vamos a tener que diferenciar las tablas de traducción de cada uno de los modelos, ya que cada uno tiene su propio vocabulario.

In [31]:
import random
import tensorflow as tf

modelo_abascal = tf.keras.models.load_model('./modelos/abascal.keras')
modelo_sanchez = tf.keras.models.load_model('./modelos/sanchez.keras')
modelo_casado = tf.keras.models.load_model('./modelos/casado.keras')

modelos = [modelo_abascal, modelo_sanchez, modelo_casado]
politicos = ["Abascal", "Sanchez", "Casado"]

ultima_palabra = u"¿Que opina del señor Abascal?"
turno = random.randint(0,2)

for i in range(10):
    turno = (turno + 1) % 3
    ultima_palabra = generate_text(modelos[turno], start_string=ultima_palabra)
    print(f"{politicos[turno]}: {ultima_palabra}")

    

Abascal:  Por eso tenga cuidado porque también ha perdido la de la luz. Oiga, por cierto, si usted pedía dimisiones cuando la luz subía un 8 %, ¿por qué no dimo dictaron la presidenta de la Comisión y el presidente del Consejo Europeo, el comisario europeo de la señora Calvo, a la señora Montero y al señor Planas. Usted tiene un secretario de Estado de Turiste esta gestión, y para coordinar bien las ayudas sociales, las medidas como la renta mínima, que sí le tenga que pagar un 30 % de luz? ¿No va a demo
Sanchez: cracia española, que estamos sufriendo una oposición que no es solo después de sus palabras están sus aciendo con el Gobierno.  Si usted quiere llegar a acuerdos empecemos por lo básico, por lo que entiendo que se destruyan empleos y también empresas. Pero, fíjese, señor Casado, gracias a las medidas que nos trajo protección gracias al descubrimiento de la valunta. Este es un año, después de siempre los ciudadanos, ni de la protección y clasificación, los órganos y las autorid